# notebooks



## introduction





A couple of rules:
- all public functions are made public using module.export or global namespace
- use describe( test blocks to describe parsing and replacement information for the functions it is testing, i.e. function utility(root) would have a describe block describe('utility(root)') neatly formated and parsed for calendar commands
- every file is one feature
- every cell or export should be a single purpose component with proper includes/dependencies
- every cell must be under 100 lines
- every function must be testable in isolation
- every function should be made accesible from the command line using module.exports
- every function should be runnable from notebooks using typeof $$ !== 'undefined'


```javascript

if(typeof $$ !== 'undefined') {
    $$.async();
    exportAndDeploy('../Frameworks/zuora to eloqua.ipynb')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}

```
- every module should have a markdown title, at least one question that the code intends to answer (how to?), includes at least (where name is the name of any function in the code block):


```javascript

...
function <name> () {
...
}
...
exports = <name>
...

```

 - eliminate circular dependencies the same way unintended recursion is avoided, create a condition:
 
ModuleA.js
```
var funcB = require('ModuleB.js');
```

ModuleB.js
```
var funcA = require('ModuleA.js');
```

Becomes:

ModuleA.js
```
var funcB = require('ModuleB.js');
```

ModuleB.js
```
if(typeof funcA === 'undefined') {
    var funcA = require('ModuleA.js');
}
```

 - parameters are listed in most specific left to least specific right, i.e. function(filter, context) would mean filter is only used for this function, whereas context may be passed in to this function as well as other functions.  "filter" is on the left because it is specifically used just for this function, context is on the right because it might contain a path on the filesystem, or some options.
 - Entry cells, i.e. cells that are intended to be called by a service, coordinate interactions between multiple services - should not contain catch blocks so that the task scheduler will fail and log the last error.


readme.md?

In [ ]:
// readme placeholder

## import notebook

### import all cell modules?

import files back in to cells? (two-way workflow)


#### the code

In [ ]:
var fs = require('fs');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Documents/jupytangular2/Utilities/.modules';
var project = PROFILE_PATH + '/Documents/jupytangular2';

var cells = glob.sync('**/cell-*', {cwd: PROJECT_PATH});
for(const c of cells) {
    const cell = path.basename(c);
    const notebook = path.basename(path.dirname(c));
    const parent = path.basename(path.dirname(path.dirname(c)));
    if(parent === 'jupytangular2') {
        continue;
    }
    const nb = JSON.parse(fs.readFileSync(path.join(
        project, parent, notebook + '.ipynb')));
    let counter = 0;
    for(const i in nb.cells) {
        if(!nb.cells.hasOwnProperty(i)) {
            continue;
        }
        // TODO: reimport all cells
        if(nb.cells[i].cell_type === 'code') {
            if(cell === 'cell-' + counter + '.js') {
                nb.cells[i].source = fs.readFileSync(path.join(PROJECT_PATH, c)).toString().split('\n');
            }
            counter++;
        }
    }
    fs.writeFileSync(path.join(project, parent, notebook + '.ipynb'),
                     JSON.stringify(nb, null, 2));
}


### TODO: import notebooks from stack overflow links


### TODO: import notebooks from other sources


## export notebook



### automatically replace notebook dependency



#### replace core requirement?



In [ ]:
var importer = require('../Core')
var {selectAst} = importer.import('select code tree')
var {transpile, remove} = importer.import('transpile code')

var CORE_DECLARE = `//VariableDeclaration[
    .//*[@type="Literal" and contains(@value, "Core")]
]`
var IMPORTER = `${CORE_DECLARE}/*[@type="Identifier"]/@name`
var IMPORTER_CALLS = `//CallExpression[./*[
    contains(@type, "Member") and ./*[@type="Identifier" and @name=${IMPORTER}]
]]`

// TODO: generalize this and make a tool
function replaceProperty(ctx) {
    var usage = selectAst(`./*[@parent-attr="callee"]`, ctx);
    var property = selectAst(`./*/*[@parent-attr="property"]`, ctx);
    if(selectAst(`./@name`, property) === 'import') {
        property.setAttribute('name', 'importNotebook')
    }
    usage.replaceWith(property)
    property.setAttribute('parent-attr', 'callee')
}

function replaceCore(code) {
    return transpile([
        [IMPORTER_CALLS, replaceProperty],
        [CORE_DECLARE, remove],
    ], code)
}

module.exports = {
    replaceCore
}


#### replace notebook import?



In [ ]:
var importer = require('../Core')
var {selectAst} = importer.import('select code tree')
var {transpile} = importer.import('transpile code')
var niceName = importer.import('rename cell to nice name')
var {htmlToTree} = importer.import('html to tree')

var IMPORT_CALLS = `//CallExpression[
./*/Identifier[@name="import" or @name="importNotebook"]]`

function getImportTemplate(imports) {
    throw new Error('TODO: multiple import template')
}

function replaceImport(ctx) {
    var str = selectAst([`./Literal/@value`], ctx)[0]
    if(!str) {
        throw new Error(`Error: dynamic include ${ctx.ownerDocument.toString(ctx)}, TODO: include Core`)
    }
    var result = importer.interpret(str)
    if(Array.isArray(result)) {
        template = getImportTemplate(imports)
    } else {
        template = selectAst([`//CallExpression`], `require("./${niceName(result)}")`)[0]
    }
    var parent = ctx.parentNode
    parent.replaceChild(template, ctx)
    template.setAttribute('parent-attr', 'init')
}

function replaceImports(code) {
    return transpile([
        [IMPORT_CALLS, replaceImport]
    ], code)
}

module.exports = {
    replaceImports
}


#### test notebook imports



In [8]:
var importer = require('../Core')
var {
    replaceImports, replaceCore
} = importer.import(['replace notebook import', 'replace core requirement'])
var {selectAst} = importer.import('select code tree')
var {htmlToTree} = importer.import('html to tree')

var code = `
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
`

if(typeof $$ != 'undefined') {
    $$.mime({'text/plain': replaceCore(code).ownerDocument.toString()})
    
    /*
    expected output 
var getArrayAST = importer.import('get ast path array');
*/
    
}



importing replace notebook import,replace core requirement - 2 cells - notebook.ipynb[5],notebook.ipynb[4]


var getArrayAST = importer.import('get ast path array');

#### add missing imports?

automatic jupyter dependency injector?

TODO: use an extension instead of import() https://github.com/bahmutov/node-hook


In [ ]:
var importer = require('../Core')
var {transpile} = importer.import('transpile code')
var {selectAst} = importer.import('select code tree')
var niceName = importer.import('rename cell to nice name')
var exportsCache = importer.import('exports cache')

var GLOBAL_CALLS = `//CallExpression[
not(./parent::MemberExpression)
and not(//*[contains(@type, "Declar")]/Identifier/@name=./Identifier/@name)
]`

/*
[

and not(//MemberExpression/Identifier/@name=./Identifier/@name)
]
*/

var notebookExports;

function addImport(ctx) {
    var id = selectAst(`./Identifier/@name`, ctx)
    var file = exportsCache.filter(e => e[2].includes(id))
    if(file.length === 1) {
        var body = selectAst([`//Program`], ctx)[0]
        var include = selectAst([`//Program/*`],
                                `var ${id} = importNotebook("${file[0]}")`)[0]
        body.insertBefore(include, body.childNodes[0] || null)
    } else if (file.length > 1) {
        throw new Error(`undefined ${id}, couldn't import ${JSON.stringify(file)}`)
    }
}

function addImports(code) {
    return transpile([
        [GLOBAL_CALLS, addImport]
    ], code)
}

module.exports = {
    addImports
}


#### test add missing imports?

test adding a missing import on its own code


In [ ]:
var importer = require('../Core')
var {addImports} = importer.import('add missing imports')

var code = `
var importer = require('../Core');

addImports('some code')

`

if(typeof $$ != 'undefined') {
    $$.mime({'text/plain': addImports(code)})
    
    /*
    expected output 
var addImports = importer.import('add missing imports')
*/
    
}



#### TODO: automatically add exports to cells

For functions that are used elsewhere.


### export cells

TODO: automatically find missing imports from distrib/Github/3rd party sources in any language?



#### the code

export notebook?

TODO: remove and rewrite all this with new transpile calls from above.


In [ ]:
var fs = require('fs')
var path = require('path')
var mkdirpSync = importer.import('mkdirp')

var importer = require('../Core')
var authorTemplate = importer.import('authoring header template')
var {
    replaceImports, replaceCore
} = importer.import(['replace notebook imports', 'replace core requirement'])
var getImports = importer.import('get imports')
var {fixImports} = importer.import('fix project paths')
var delintCode = importer.import('delint notebooks')
var niceName = importer.import('rename cell to nice name')
var {matchFilename} = importer.import('match filename')

function getImportsRecursively(searches) {
    if(typeof searches === 'string') {
        searches = [searches]
    }
    const processed = []
    const allCells = []
    const pending = importer.interpret(searches)
    while(pending.length > 0) {
        var cell = pending.pop()
        console.log(cell.id)
        processed.push(cell.id)
        allCells.push(cell)
        if(cell.code.length > 10000 || cell.filename.includes('cache.ipynb')) continue
        var imports = getImports(cell.code)
        console.log(JSON.stringify(imports))
        imports.forEach(search => {
            try {
                var cells = importer.interpret([search])
                cells.forEach(c => {
                    if(!processed.includes(c.id))
                        pending.push(c)
                })
            } catch (e) {console.log(e)}
        })
    }
    return allCells
}

// searches are the top level cells starting the import tree
function exportNotebook(searches, projectOutput, matchOutput) {
    projectOutput = projectOutput || process.env.EXPORT_PATH
        || path.join(path.resolve(__dirname), '../.functions');

    const nextImports = []
    const cells = getImportsRecursively(searches)
    cells.forEach((cell, i) => {
        var exportedCode
        assert(!niceName(cell).match(/^\./),
               `No filename ${cell.id} ${cell.questions}!`)
        // some special exceptions with file-naming
        if(cell.name === 'readme.md') {
            exportedCode = cell.markdown
        } else if (cell.language === 'javascript') {
            try {
                exportedCode = replaceImports(cell.code).ownerDocument.toString()
                debugger
                exportedCode = replaceCore(exportedCode).ownerDocument.toString()
                const delinted = delintCode(exportedCode)[0]
                exportedCode = delinted.fixed || delinted.code
            } catch (e) {
                console.log(exportedCode)
                throw new Error(`Error exporting ${cell.id}: ${e.message}`)
            }
        } else {
            exportedCode = cell.code
        }
        outputExport(exportedCode, cell, projectOutput, matchOutput)
    })
    
    return fixImports(projectOutput)
    // TODO: output packed cells and cache
    // TODO: zip and upload to AWS
}

function outputExport(exportedCode, cell, projectOutput, matchOutput) {
    // emit the file in every location request
    matchFilename(niceName(cell), matchOutput, projectOutput).forEach(filename => {
        console.log(`emitting ${filename}`);
        // create directory if needed
        mkdirpSync(path.dirname(filename));
        // add a code block header
        if(path.extname(filename) == '.js' || path.extname(filename) == '.cs') {
            exportedCode = authorTemplate(cell.markdown, exportedCode)
        }
        fs.writeFileSync(filename, exportedCode);
    })
}

module.exports = exportNotebook;


#### inject cells in to notebooks when built with webpack?

1) Instead of loading cache from file system, load it from a combined JSON.

2) Use the ```require``` method for the included features instead of Node vm.


In [ ]:
var path = require('path');
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
var {importsTemplate, cachedTemplate} = importer.import('include require like import');

// TODO: this should be a test in the Core notebook
// inject cellCache and cellIds loaded in to the interpreter so notebooks don't need to be uploaded,
//   and webpack can tree-shake on functions already packed
function injectImports(ast, allImports) {
    var cacheBody = getArrayAST(`//*[/*/ExpressionStatement/CallExpression[
/Identifier[@name == 'cacheAll']]]`, ast)[0];
    var firstCache = getArrayAST(`//ExpressionStatement[/CallExpression[
/Identifier[@name == 'cacheAll']]]`, cacheBody)[0];
    var cacheCode = getArrayAST('*', cachedTemplate(cellsToNotebook(allImports)))[0];
    cacheBody.body.splice(cacheBody.body.indexOf(firstCache), 1, ...cacheCode.body);
    var runContext = getArrayAST(`//AssignmentExpression[
//Identifier[@name == 'runInNewContext']]`, ast)[0];
    var addImports = Object.values(allImports).map(i => i.id).filter((i, j, arr) => arr.indexOf(i) === j)
    var requireCode = getArrayAST('*', importsTemplate(addImports))[0].body[0];
    requireCode.expression.right = runContext.right;
    runContext.right = requireCode;
}

module.exports = injectImports;


### couple of tools


#### get cell extension?

Convert the notebook type to a file extension.


In [ ]:
// TODO: replace with a library?

// TODO: move into interpret notebook cell?

function getExtension(cell, notebook) {
    var extension;
    if(cell.cell_type === 'markdown') {
        extension = '.md';
    } else if(cell.cell_type === 'raw') {
        extension = '.txt';
    } else if(cell.language === 'javascript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)) {
            extension = '.spec.js'
        } else {
            extension = '.js';
        }
    } else if(cell.language === 'powershell') {
        extension = '.ps1';
    } else if(cell.language === 'csharp') {
        extension = '.cs';
    } else if(cell.language === 'python') {
        extension = '.py';
    } else if(cell.language === 'typescript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)
            || (cell.questions[0] || '').includes('test')) {
            extension = '.spec.ts'
        } else {
            extension = '.ts';
        }
    } else if(cell.language === 'bash') {
        extension = '.sh';
    } else {
        throw 'unknown language or cell type: '
            + (cell.filename || notebook)
            + ' (' + cell.cell_type + ',' + cell.language + ')';
    }
    return extension;
}
module.exports = getExtension;


#### rename cell to a nice name?

TODO: move this to Core import notebook?

In [ ]:
var importer = require('../Core');
var getExtension = importer.import('cell extension')

// get previous markdown and extract name that leads back to the current cell
function niceName(cell) {
    var name = cell.questions[0].replace('?', '')
        .replace(/[^a-z0-9]+|\btest\b/igm, ' ')
        .trim()
        .replace(/\s+/igm, '-')
        + getExtension(cell);
    if(name.toLowerCase().includes('readme')) {
        name = 'readme.md';
    }
    if(name.toLowerCase().includes('package-json')) {
        name = 'package.json';
    }
    return name;
}

module.exports = niceName;


#### authoring header template?


In [ ]:

// TODO: move this heading authoring template to utility function
function authorTemplate(markdown, code) {
    return `
/**
 * Written by Brian Cullinan, exported using magic.
 * Copyright (c) ${(new Date()).getFullYear()} by Brian Cullinan, All rights reserved.
 *
${((markdown || '') + '').split('\n').map(l => ' * ' + l).join('\n')}
 *
 **/

${code}`;
}

module.exports = authorTemplate;

In [ ]:
console.log(global)